In [43]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, Variable, optimizers, serializers, utils
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L

import sys
# the mock-0.3.1 dir contains testcase.py, testutils.py & mock.py
sys.path.append('/Users/roberm3/learn/chainer-1.8.1/examples/mnist')
import data


In [44]:
mnist = data.load_mnist_data()

Done
Done
Done
Done
Converting training data...
Done
Converting test data...
Done
Save output...
Done
Convert completed


In [68]:
#scale pixel values to 0 or 1
x_all = mnist['data'].astype(np.float32) / 255
y_all = mnist['target'].astype(np.int32)
x_train, x_test = np.split(x_all, [60000])
y_train, y_test = np.split(y_all, [60000])


In [69]:
class MLP(Chain):
    def __init__(self):
        super(MLP, self).__init__(
            l1=L.Linear(784,100),
            l2=L.Linear(100,100),
            l3=L.Linear(100,10))
        
    def __call__(self,x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        y = self.l3(h2)
        return y
    
        

In [70]:
class Classifier(Chain):
     def __init__(self, predictor):
         super(Classifier, self).__init__(predictor=predictor)

     def __call__(self, x, t):
         y = self.predictor(x)
         self.loss = F.softmax_cross_entropy(y, t)
         self.accuracy = F.accuracy(y, t)
         return self.loss

In [80]:
model = L.Classifier(MLP())
optimizer = optimizers.SGD()
optimizer.setup(model)


batchsize = 100
datasize = 60000
for epoch in range(20):
    print('epoch %d' % epoch)
    indexes = np.random.permutation(datasize)
    for i in range(0,datasize, batchsize):
        x = Variable(x_train[indexes[i : i + batchsize]])
        t = Variable(y_train[indexes[i : i + batchsize]])
        optimizer.update(model, x, t)
        
    

epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19


In [82]:
sum_loss, sum_accuracy = 0, 0
for i in range(0,10000,batchsize):
    x = Variable(x_test[i: i + batchsize])
    t = Variable(y_test[i: i + batchsize])
    loss = model(x,t)
    sum_loss += loss.data * batchsize
    sum_accuracy += model.accuracy.data * batchsize
    
mean_loss = sum_loss / 10000
mean_accuracy = sum_accuracy / 10000

print("Mean Loss: {}".format(mean_loss))
print("Mean Accu: {}".format(mean_accuracy))

Mean Loss: 0.155032213274
Mean Accu: 0.95490000546


In [79]:
range(0,10,2)

[0, 2, 4, 6, 8]